In [42]:
from __future__ import division
import glob
from datetime import datetime, date, timedelta
import os
import itertools
import re

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from pandas import DataFrame, Series
import pandas as pd

%matplotlib inline
matplotlib.rcParams['svg.fonttype'] = 'none'

pd.options.display.max_rows = 150

In [43]:
path = os.getcwd()
files = os.listdir(path)
files

['get_all_data.ipynb',
 'KHV_2012.xlsx',
 'khv2000.pdf',
 'myvenv',
 'KHV_2003.xls',
 'KHV_2015.xlsx',
 'KHV_2011.xls',
 'README.md',
 'KHV_2007.xls',
 'REHA_95.XLS',
 'KHV_2009.xls',
 'geocoding.ipynb',
 '.~lock.KHV_2004.xls#',
 'KHV_2004.xls',
 'KHV_2014.xlsx',
 'todo_KHV_1990.xlsx_todo',
 '.git',
 'KHV_2005.xls',
 'KHV_2006.xls',
 'KHV_1992.xls',
 'KHV_2008.xls',
 '.~lock.KHV_2003.xls#',
 'KHV_2013.xlsx',
 'KHV_2010.xls',
 'KHV_1995.XLS',
 'KHV_1990.pdf',
 '.ipynb_checkpoints',
 'concat_data.ipynb',
 'map1.html',
 'todo_KHV_2000.xlsx_todo',
 '.vscode',
 'krankenhaus_privatisierung.ipynb',
 '.gitgnore']

In [44]:
files_xls = [f for f in files if f[-3:] == 'xls']
files_xlsx = [f for f in files if f[-4:] == 'xlsx']
files = files_xls + files_xlsx
files.sort()
files

['KHV_1992.xls',
 'KHV_2003.xls',
 'KHV_2004.xls',
 'KHV_2005.xls',
 'KHV_2006.xls',
 'KHV_2007.xls',
 'KHV_2008.xls',
 'KHV_2009.xls',
 'KHV_2010.xls',
 'KHV_2011.xls',
 'KHV_2012.xlsx',
 'KHV_2013.xlsx',
 'KHV_2014.xlsx',
 'KHV_2015.xlsx']

In [22]:
files = ['KHV_2013.xlsx']

In [39]:
df = pd.DataFrame
regex = r"\d{4}"
df = pd.DataFrame

for file in files:
    # sheet_name = re.search(regex_sheet, )
    xls = pd.ExcelFile(file)
    sheet_name = [x for x in xls.sheet_names if x.startswith('KHV')]
    print(file, sheet_name)
    excel = xls.parse(sheet_name)
    
    year = re.search(regex, file).group()
    # data['Jahr'] = year
    df = pd.DataFrame(excel, index=[0])

KHV_2013.xlsx ['KHV_2013']


ValueError: Shape of passed values is (71, 1), indices imply (1, 1)

In [ ]:
df.head()

In [35]:
filenames = [('KHV_', range(2009, 2012)),
             ('data/AWB KBV Meldungen und Abschlüsse 2012 - 2014.xlsx', (2012, 2013, 2014))]


def read_excel(filename, years, kind='update', needle='Meldungen'):
    xl_file = pd.ExcelFile(filename)
    sheet_names = xl_file.sheet_names

    for year in years:
        sheet_name = [x for x in sheet_names if str(year) in x and needle in x][0]
        print(filename, sheet_name)
        df = xl_file.parse(sheet_name)
        # Make index into row number column
        df = df.reset_index()
        df['year'] = year
        df['row_type'] = kind
        df = df.rename(columns=dict([(x, x.strip()) for x in df.columns if x.strip() != x]))
        # Consolidate column names 
        df = df.rename(columns={u'Präparat': u'Präparatname',
                                u'Präparat/Titel der Anwendung': u'Präparatname',
                                u'gemeldet am': 'DatumErstanzeige',
                                u'Datum Erstanzeige': 'DatumErstanzeige',
                                u'Beobachtugsplan vorliegend': 'Beobachtungsplan vorliegend',
                                u'Ärzte gemeldet': u'gemeldete Ärzte',
                                u'Anzahl teilnehmende Ärzte (wenn angegeben)': u'gemeldete Ärzte',
                                u'Anzahl der beobachtenden Ärzte': u'beobachtende Ärzte',
                                'index': 'row_number'
                               })
        if 'DatumErstanzeige' not in df.columns:
            df = df.rename(columns={'Eingang': 'DatumErstanzeige'})
        # Fix date columns
        date_cols = list(df.columns[df.columns.str.startswith('Datum')])
        for x in date_cols:
            df['dt_%s' % x] = pd.to_datetime(df[x], errors='coerce')
        df[u'Präparatname'] = df[u'Präparatname'].str.strip()
        # Remove entries with empty drug name
        df = df[df[u'Präparatname'].notnull()]
        yield df